## Загрузка прочтений

Сначала загружу SRA Toolkit с сайта NCBI:

In [1]:
!wget https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/3.0.0/sratoolkit.3.0.0-ubuntu64.tar.gz
!tar -xvzf sratoolkit.3.0.0-ubuntu64.tar.gz

--2023-12-17 11:24:09--  https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/3.0.0/sratoolkit.3.0.0-ubuntu64.tar.gz
Resolving ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)... 130.14.250.11, 130.14.250.12, 2607:f220:41e:250::13, ...
Connecting to ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)|130.14.250.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85777118 (82M) [application/x-gzip]
Saving to: ‘sratoolkit.3.0.0-ubuntu64.tar.gz’

sratoolkit.3.0.0-ub 100%[===================>]  81.80M   116MB/s    in 0.7s    

2023-12-17 11:24:09 (116 MB/s) - ‘sratoolkit.3.0.0-ubuntu64.tar.gz’ saved [85777118/85777118]

sratoolkit.3.0.0-ubuntu64/
sratoolkit.3.0.0-ubuntu64/bin/
sratoolkit.3.0.0-ubuntu64/bin/abi-dump
sratoolkit.3.0.0-ubuntu64/bin/abi-dump.3
sratoolkit.3.0.0-ubuntu64/bin/abi-load
sratoolkit.3.0.0-ubuntu64/bin/abi-load.3
sratoolkit.3.0.0-ubuntu64/bin/align-info
sratoolkit.3.0.0-ubuntu64/bin/align-info.3
sratoolkit.3.0.0-ubuntu64/bin/bam-load
sratoolki

Нам необходимо загрузить прочтения при помощи программы <code>fasterq-dump</code> из SRA Toolkit. Внутри Google Colab настроить SRA Toolkit правильно крайне сложно, поэтому необходимо будет выполнить некоторую странную процедуру: специально вызвать Segmentation fault у инструмента <code>vdb-config</code>.

In [2]:
!sratoolkit.3.0.0-ubuntu64/bin/vdb-config -i

2023-12-17T11:24:16 vdb-config.3.0.0 fatal: SIGNAL - Segmentation fault 


## Контроль качества прочтений

### FastQC

Оценку качества буду проводить с помощью <code>FastQC</code>. Для этого скачаю программу

In [3]:
!wget https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.9.zip
!unzip fastqc_v0.11.9.zip

--2023-12-17 11:24:16--  https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.9.zip
Resolving www.bioinformatics.babraham.ac.uk (www.bioinformatics.babraham.ac.uk)... 149.155.133.4
Connecting to www.bioinformatics.babraham.ac.uk (www.bioinformatics.babraham.ac.uk)|149.155.133.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10249221 (9.8M) [application/zip]
Saving to: ‘fastqc_v0.11.9.zip’

fastqc_v0.11.9.zip  100%[===================>]   9.77M  10.5MB/s    in 0.9s    

2023-12-17 11:24:17 (10.5 MB/s) - ‘fastqc_v0.11.9.zip’ saved [10249221/10249221]

Archive:  fastqc_v0.11.9.zip
  inflating: FastQC/cisd-jhdf5.jar   
   creating: FastQC/Configuration/
  inflating: FastQC/Configuration/adapter_list.txt  
  inflating: FastQC/Configuration/contaminant_list.txt  
  inflating: FastQC/Configuration/limits.txt  
  inflating: FastQC/fastqc           
  inflating: FastQC/fastqc_icon.ico  
   creating: FastQC/Help/
   creating: FastQC/Help/1 Introductio

Затем сделаю скрипт <code>fastqc</code> выполняемым. В результате выполнения анализа качества прочтений буду класть результаты и все репорты в папку <code>qc</code>, которую создаю.

In [4]:
!chmod +x FastQC/fastqc
!mkdir qc

### MultiQC

Теперь установлю программу MultiQC. Она необходима для того, чтобы комбинировать несколько различных отчётов в один. Запустить её можнл, дав путь к директории с отчётами FastQC (<code>!multiqc qc</code>), и посмотрев на .html-отчёт, который будет получен в результате запуска.

In [5]:
!pip3 install multiqc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.2 MB/s eta 0:00:00
  Created wheel for spectra: filename=spectra-0.0.11-py3-none-any.whl size=17467 sha256=3b60c18594dde877148fdebbb98e1b536aad083f075313ea40287ec7e183bf40
  Stored in directory: /root/.cache/pip/wheels/bc/27/ec/cddaa9489a49ddcf702e1f4dc1791db28a3b0121e1ff254f97
  Created wheel for colormath: filename=colormath-3.0.0-py3-none-any.whl size=39405 sha256=328f002e77e607a77945188dc758f5cbc30ea7b7e25f8949d2d60c827e8dae11
  Stored in directory: /root/.cache/pip/wheels/ab/b3/4d/c0738759c25a1df01958068f162cf2a9dc3ab1da8b972cfcfc
Successfully built spectra colormath


## Выполнение задания с помощью Nexflow

NextFlow &mdash; это специальный язык, который позволяет запускать целые пайплайны из нескольких последовательно выполняемых программ, следя за тем, чтобы каждая стадия выполнялась строго одна за одной.

In [6]:
!curl -fsSL https://get.nextflow.io | bash

CAPSULE: Downloading dependency org.multiverse:multiverse-core:jar:0.7.0
CAPSULE: Downloading dependency org.apache.ivy:ivy:jar:2.5.1
CAPSULE: Downloading dependency org.codehaus.groovy:groovy:jar:3.0.19
CAPSULE: Downloading dependency commons-io:commons-io:jar:2.11.0
CAPSULE: Downloading dependency com.google.guava:listenablefuture:jar:9999.0-empty-to-avoid-conflict-with-guava
CAPSULE: Downloading dependency io.nextflow:nf-httpfs:jar:23.10.0
CAPSULE: Downloading dependency com.beust:jcommander:jar:1.35
CAPSULE: Downloading dependency io.nextflow:nf-commons:jar:23.10.0
CAPSULE: Downloading dependency jline:jline:jar:2.9
CAPSULE: Downloading dependency org.codehaus.groovy:groovy-xml:jar:3.0.19
CAPSULE: Downloading dependency org.slf4j:log4j-over-slf4j:jar:2.0.7
CAPSULE: Downloading dependency com.github.zafarkhaja:java-semver:jar:0.9.0
CAPSULE: Downloading dependency javax.mail:mail:jar:1.4.7
CAPSULE: Downloading dependency com.fasterxml.jackson.core:jackson-annotations:jar:2.14.2
CAPSU

# Установка дополнительных пакетов для работы скрипта

In [19]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

--2023-12-17 11:42:45--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120986213 (115M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>] 115.38M   161MB/s    in 0.7s    

2023-12-17 11:42:46 (161 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [120986213/120986213]

PREFIX=/usr/local
Unpacking payload ...
                                                                                        
Installing base environment...





Preparing transaction: - \ | / - done
Executing transaction: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | 

In [136]:
!conda install -c bioconda bwa -y
!conda install -c bioconda varscan -y

Channels:
 - bioconda
 - defaults
Platform: linux-64
Solving environment: / - done

# All requested packages already installed.

Channels:
 - bioconda
 - defaults
Platform: linux-64
Solving environment: \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - varscan


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    dbus-1.13.18               |       hb2f20db_0         504 KB
    expat-2.5.0                |       h6a678d5_0         172 KB
    freetype-2.12.1            |       h4a9f257_0         626 KB
    glib-2.69.1                |       he621ea3_2         1.9 MB
    libpng-1.6.39              |       h5eee18b_0         304 KB
    libxcb-1.15                |       h7f8727e_0         505 KB
    openjdk-11.0.13            |       h87a67e3_0       341.0 MB
    pcre-8.45                  |       h295c915_0         207 KB
    varscan-2.4.6 

In [135]:
!sudo apt install bowtie2 samtools

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libhts3 libhtscodecs2
Suggested packages:
  bowtie2-examples cwltool
The following NEW packages will be installed:
  bowtie2 libhts3 libhtscodecs2 samtools
0 upgraded, 4 newly installed, 0 to remove and 24 not upgraded.
Need to get 2,131 kB of archives.
After this operation, 6,005 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libhtscodecs2 amd64 1.1.1-3 [53.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libhts3 amd64 1.13+ds-2build1 [390 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 samtools amd64 1.13-4 [520 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 bowtie2 amd64 2.4.4-1 [1,169 kB]
Fetched 2,131 kB in 1s (2,134 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the

In [7]:
text_file = open("my_pipeline.nf", "w")
text_file.write('''nextflow.enable.dsl=2
params.ref='results/sequence.fasta'
params.results_dir = "results/"
seq_file = file(params.ref)

SRA_list = params.SRA.split(",")

log.info ""
log.info "  Q U A L I T Y   C O N T R O L  "
log.info "================================="
log.info "SRA number         : ${SRA_list}"
log.info "Results location   : ${params.results_dir}"

process DownloadFastQ {
  publishDir "${params.results_dir}"

  input:
    val sra

  output:
    path "${sra}/*"

  script:
    """
    /content/sratoolkit.3.0.0-ubuntu64/bin/fasterq-dump ${sra} -O ${sra}/
    """
}

process QC {
  input:
    path x

  output:
    path "qc/*"

  script:
    """
    mkdir qc
    /content/FastQC/fastqc -o qc $x
    """
}

process MultiQC {
  publishDir "${params.results_dir}"

  input:
    path x

  output:
    path "multiqc_report.html"

  script:
    """
    multiqc $x
    """
}

process index {
  input:
    file(seq_file)

  script:
    """
    /usr/local/bin/bwa index ${seq_file}
    """
}

workflow {
  ref_ch=Channel.fromPath(params.ref)
  index(ref_ch)
  data = Channel.of( SRA_list )
  DownloadFastQ(data)
  QC( DownloadFastQ.out )
  MultiQC( QC.out.collect() )
}
''')
text_file.close()

Теперь для того, чтобы обеспечить параллельный запуск сразу нескольких процессов одним и тем же пайплайном, необходимо распарсить аргумент, который мы подаём в переменную <code>SRA</code> как список, после чего запустить пайплайн созданного файла <code>my_pipeline.nf</code> со следующим содержанием:
```
params.results_dir = "results/"
SRA_list = params.SRA.split(",")

log.info ""
log.info "  Q U A L I T Y   C O N T R O L  "
log.info "================================="
log.info "SRA number         : ${SRA_list}"
log.info "Results location   : ${params.results_dir}"

process DownloadFastQ {
  publishDir "${params.results_dir}"

  input:
    val sra

  output:
    path "${sra}/*"

  script:
    """
    /content/sratoolkit.3.0.0-ubuntu64/bin/fasterq-dump ${sra} -O ${sra}/
    """
}

process QC {
  input:
    path x

  output:
    path "qc/*"

  script:
    """
    mkdir qc
    /content/FastQC/fastqc -o qc $x
    """
}

process MultiQC {
  publishDir "${params.results_dir}"

  input:
    path x

  output:
    path "multiqc_report.html"

  script:
    """
    multiqc $x
    """
}

workflow {
  data = Channel.of( SRA_list )
  DownloadFastQ(data)
  QC( DownloadFastQ.out )
  MultiQC( QC.out.collect() )
}
```
Для примера возьму несколько файлов (`SRR1705851,SRR1705858,SRR1705859,SRR1705860`).

In [8]:
!./nextflow run my_pipeline.nf -with-report nextflow_report.html -with-dag flowchart.pdf --SRA SRR1705851,SRR1705858,SRR1705859,SRR1705860

N E X T F L O W  ~  version 23.10.0
Launching `my_pipeline.nf` [mad_payne] DSL2 - revision: 0b65247c94

  Q U A L I T Y   C O N T R O L  
SRA number         : [SRR1705851, SRR1705858, SRR1705859, SRR1705860]
Results location   : results/
[-        ] process > DownloadFastQ -
[-        ] process > QC            -
[-        ] process > MultiQC       -

[-        ] process > DownloadFastQ -
[-        ] process > QC            -
[-        ] process > MultiQC       -

executor >  local (2)
[a1/dc9208] process > DownloadFastQ (2) [  0%] 0 of 4
[-        ] process > QC                -
[-        ] process > MultiQC           -

executor >  local (2)
[a1/dc9208] process > DownloadFastQ (2) [  0%] 0 of 4
[-        ] process > QC                -
[-        ] process > MultiQC           -

executor >  local (3)
[fd/308816] process > DownloadFastQ (3) [  0%] 0 of 4
[-        ] process > QC                -
[-        ] process > MultiQC           -

executor >  local (3)
[fd/308816] process > Downl

Посмотрите на содержимое файов `flowchart.pdf` и `nextflow_report.html`.

In [111]:
!bwa index sequence.fasta

[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.00 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index sequence.fasta
[main] Real time: 0.037 sec; CPU: 0.007 sec


In [109]:
!./nextflow run my_new_pipeline.nfя

N E X T F L O W  ~  version 23.10.0
Launching `my_new_pipeline.nf` [compassionate_marconi] DSL2 - revision: b2f2aeab01
[-        ] process > index -

[-        ] process > index [  0%] 0 of 1

executor >  local (1)
[ae/2f0d36] process > index (1) [100%] 1 of 1 ✔

executor >  local (1)
[ae/2f0d36] process > index (1) [100%] 1 of 1 ✔



In [20]:
!conda install -c bioconda bwa -y

Channels:
 - bioconda
 - defaults
Platform: linux-64
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - bwa


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bwa-0.7.17                 |       h5bf99c6_8         608 KB  bioconda
    certifi-2023.11.17         |  py311h06a4308_0         159 KB
    conda-23.11.0              |  py311h06a4308_0         1.3 MB
    distro-1.8.0               |  py311h06a4308_0          37 KB
    gdbm-1.18                  |       hd4cb3f1_4         194 KB
    menuinst-2.0.1             |  py311h06a4308_1         164 KB
    perl-5.34.0                |       h5eee18b_2        12.4 MB
    platformdirs-3.10.0        |  py311h06a4308_0          37 KB
    ------------------------------------------------------------
                                           Total:        14.8 MB

The following 

In [ ]:
!mkdir ref
!bwa index sequence.fasta
!bwa mem sequence.fasta /content/results/SRR1705851/SRR1705851.fastq > alignment.sam

mkdir: cannot create directory ‘ref’: File exists
[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.00 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index sequence.fasta
[main] Real time: 0.036 sec; CPU: 0.006 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 68388 sequences (10000129 bp)...
[M::process] read 67628 sequences (10000233 bp)...
[M::mem_process_seqs] Processed 68388 reads in 3.250 CPU sec, 3.188 real sec
[M::process] read 67698 sequences (10000046 bp)...
[M::mem_process_seqs] Processed 67628 reads in 2.272 CPU sec, 2.346 real sec
[M::process] read 67652 sequences (10000169 bp)...
[M::mem_process_seqs] Processed 67698 reads in 2.469 CPU sec, 2.302 real sec
[M::process] read 68072 sequences (10000295 bp)...
[M::mem_process_seqs] Processed 67652 reads in 2.

In [ ]:
!bwa mem GCF_000005845.2_ASM584v2_genomic.fna trimmed_20.paired.R1.fastq trimmed_20.paired.R2.fastq > alignment.sam


In [137]:
!which samtools


/usr/bin/samtools


head: cannot open 'results//.fastq' for reading: No such file or directory


In [153]:
text_file = open("my_pipeline_test.nf", "w")
text_file.write('''nextflow.enable.dsl=2
params.ref='results/sequence.fasta'
params.refbwt='results/sequence.fasta.bwt'
params.fastq_dir = "results/"
params.results_dir = "results/"
seq_file = file(params.ref)

SRA_list = params.SRA.split(",")

log.info ""
log.info "  Q U A L I T Y   C O N T R O L  "
log.info "================================="
log.info "SRA number         : ${SRA_list}"
log.info "Results location   : ${params.results_dir}"

process DownloadFastQ {
  publishDir "${params.results_dir}"

  input:
    val sra

  output:
    path "${sra}/*"

  script:
    """
    /content/sratoolkit.3.0.0-ubuntu64/bin/fasterq-dump ${sra} -O ${sra}/
    """
}

process QC {
  input:
    path x

  output:
    path "qc/*"

  script:
    """
    mkdir qc
    /content/FastQC/fastqc -o qc $x
    """
}

process MultiQC {
  publishDir "${params.results_dir}"

  input:
    path x

  output:
    path "multiqc_report.html"

  script:
    """
    multiqc $x
    """
}

process index {
  input:
    file(seq_file)

  output:
    file("${seq_file}.bwt")

  script:
    """
    /usr/local/bin/bwa index ${seq_file}
    """
}

process BWA_mem {
  publishDir "${params.results_dir}"

  input:
    file ${refbwt}
    file(fastq) from results

  output:
    file("${fastq.baseName}.sam")

  script:
    """
    /usr/local/bin/bwa mem -t 4 ${bwt} ${fastq} > ${fastq.baseName}.sam
    """
}

process samtools_view {
  publishDir "${params.results_dir}"

  input:
    file(sam) from bwaMem

  output:
    file("${sam.baseName}.bam")

  script:
    """
    /usr/bin/samtools view -S -b ${sam} > ${sam.baseName}.bam
    """
}

process samtools_flagstat {
    publishDir "${params.results_dir}"

  input:
    file(bam) from samtoolsView

  output:
    path "flagstat/*"

  script:
    """
    mkdir flagstat
    /usr/bin/samtools ${mem}.unsorted.bam > flagstat/${bam.baseName}.flagstat
    """
}

process samtools_sort {
  publishDir "${params.results_dir}"

  input:
    file(bam) from samtoolsView

  output:
    file("${bam.baseName}.sorted.bam")

  script:
    """
    /usr/bin/samtools sort ${bam} > ${bam.baseName}.sorted.bam
    """
}

workflow {
  ref_ch=Channel.fromPath(params.ref)
  index(ref_ch)
  data = Channel.of( SRA_list )
  DownloadFastQ(data)
  QC( DownloadFastQ.out )
  MultiQC( QC.out.collect() )
  BWA_mem( DownloadFastQ.out )
  samtools_view( BWA_mem.out )
  samtools_flagstat( samtools_view.out )
}
''')
text_file.close()

In [142]:
!bwa mem GCF_000005845.2_ASM584v2_genomic.fna amp_res_1.fastq > alignment_1.sam
!samtools view -S -b alignment_1.sam > alignment_1.bam

[E::bwa_idx_load_from_disk] fail to locate the index files
[main_samview] fail to read the header from "alignment_1.sam".


In [154]:
!./nextflow run my_pipeline_test.nf -with-report nextflow_report.html -with-dag flowchart.pdf --SRA SRR1705851,SRR1705858,SRR1705859,SRR1705860

curl: /usr/local/lib/libcurl.so.4: no version information available (required by curl)
N E X T F L O W  ~  version 23.10.0
Launching `my_pipeline_test.nf` [chaotic_dalembert] DSL2 - revision: 5ae36ad354

  Q U A L I T Y   C O N T R O L  
SRA number         : [SRR1705851, SRR1705858, SRR1705859, SRR1705860]
Results location   : results/
[-        ] process > index         -
[-        ] process > DownloadFastQ -
[-        ] process > QC            -
[-        ] process > MultiQC       -
ERROR ~ Unknown process directive: `$`

 -- Check script 'my_pipeline_test.nf' at line: 77 or see '.nextflow.log' file for more details

